In [1]:
import tensorflow as tf
import keras
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from training.train import load_tfrecord_dataset
import os
import glob
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

2024-12-04 10:58:26.513689: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-04 10:58:26.520814: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733284706.529900  115254 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733284706.532594  115254 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-04 10:58:26.542042: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
CHECKPOINT_PATH = "/home/phatdat/Desktop/Sleep-Apnea-Detection/model/checkpoints"
RECORD_PATH = "/mnt/dat/prepped/apnea_sp02_pr_by_record"
DATASET1 = "/mnt/dat/databases/shhs/datasets/shhs1-dataset-0.21.0.csv"
DATASET2 = "/mnt/dat/databases/shhs/datasets/shhs2-dataset-0.21.0.csv"

In [3]:
trained_cps = glob.glob(os.path.join(CHECKPOINT_PATH, "test*.keras"))
trained_cps = [(int(path[path.rfind("/")+len("test_")+1:path.rfind(".keras")]), path) for path in trained_cps if "_last.keras" not in path]
trained_cps

[(255,
  '/home/phatdat/Desktop/Sleep-Apnea-Detection/model/checkpoints/test_255.keras'),
 (445,
  '/home/phatdat/Desktop/Sleep-Apnea-Detection/model/checkpoints/test_445.keras'),
 (411,
  '/home/phatdat/Desktop/Sleep-Apnea-Detection/model/checkpoints/test_411.keras'),
 (818,
  '/home/phatdat/Desktop/Sleep-Apnea-Detection/model/checkpoints/test_818.keras'),
 (698,
  '/home/phatdat/Desktop/Sleep-Apnea-Detection/model/checkpoints/test_698.keras'),
 (529,
  '/home/phatdat/Desktop/Sleep-Apnea-Detection/model/checkpoints/test_529.keras'),
 (442,
  '/home/phatdat/Desktop/Sleep-Apnea-Detection/model/checkpoints/test_442.keras'),
 (968,
  '/home/phatdat/Desktop/Sleep-Apnea-Detection/model/checkpoints/test_968.keras'),
 (743,
  '/home/phatdat/Desktop/Sleep-Apnea-Detection/model/checkpoints/test_743.keras'),
 (578,
  '/home/phatdat/Desktop/Sleep-Apnea-Detection/model/checkpoints/test_578.keras'),
 (311,
  '/home/phatdat/Desktop/Sleep-Apnea-Detection/model/checkpoints/test_311.keras'),
 (986,
  '

In [4]:
# stratify split
shhs1_csv = pd.read_csv(DATASET1, usecols=['nsrrid', 'pptid','ahi_a0h3a'])
shhs1_csv['nsrrid'] = "shhs1-" + shhs1_csv['nsrrid'].astype('str')
shhs2_csv = pd.read_csv(DATASET2, usecols=['nsrrid', 'pptid','ahi_a0h3a'], encoding_errors='replace')
shhs2_csv['nsrrid'] = "shhs2-" + shhs2_csv['nsrrid'].astype('str')

csv_df = pd.concat([shhs1_csv, shhs2_csv], ignore_index=True)
csv_df.rename(columns={'nsrrid': 'Record'}, inplace=True)

bins = [-float('inf'), 5, 15, 30, float('inf')]  # Define bins for ranges
labels = ['none', 'mild', 'moderate', 'severe']  # Corresponding labels

csv_df['ahi_label'] = pd.cut(csv_df['ahi_a0h3a'], bins=bins, labels=labels, right=False)

all_record = glob.glob(os.path.join(RECORD_PATH, "*.tfrecord"))
all_record = pd.DataFrame({"Record": [name[name.rfind("/")+1:name.rfind(".tfrecord")] for name in all_record],
                            "Path": all_record})
all_record = pd.merge(all_record, csv_df, how='left', on='Record')
all_record

,Record,Path,pptid,ahi_a0h3a,ahi_label
0,shhs2-203332,/mnt/dat/prepped/apnea_sp02_pr_by_record/shhs2...,3352.0,4.897959,none
1,shhs1-203561,/mnt/dat/prepped/apnea_sp02_pr_by_record/shhs1...,3581.0,5.115207,mild
2,shhs1-202057,/mnt/dat/prepped/apnea_sp02_pr_by_record/shhs1...,2064.0,6.486486,mild
3,shhs1-203763,/mnt/dat/prepped/apnea_sp02_pr_by_record/shhs1...,3785.0,49.793323,severe
4,shhs1-203230,/mnt/dat/prepped/apnea_sp02_pr_by_record/shhs1...,3249.0,3.916449,none
...,...,...,...,...,...
8070,shhs1-205797,/mnt/dat/prepped/apnea_sp02_pr_by_record/shhs1...,5832.0,2.099596,none
8071,shhs1-203026,/mnt/dat/prepped/apnea_sp02_pr_by_record/shhs1...,3045.0,6.241135,mild
8072,shhs1-204347,/mnt/dat/prepped/apnea_sp02_pr_by_record/shhs1...,4381.0,10.458716,mild
8073,shhs1-205387,/mnt/dat/prepped/apnea_sp02_pr_by_record/shhs1...,5422.0,44.426559,severe


In [5]:
def get_true_pred(model, dataset, batch_size, verbose=True):
    y_pred_prob = model.predict(dataset, batch_size=batch_size, verbose=verbose)
    y_true = []
    for _, y in dataset:
        y_true.append(y.numpy())
    y_true = np.vstack(y_true)
    return y_true, y_pred_prob

In [6]:
def sen_spec(y_true, y_pred_probs, threshold=None):
    """
    Calculate the geometric mean (G-mean) for a given threshold.
    """
    # Convert predicted probabilities to binary predictions based on threshold
    if threshold is None:
        y_pred = y_pred_probs
    else:
        y_pred = (y_pred_probs >= threshold).astype(int)
        
    # Compute confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    # Compute sensitivity and specificity
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    return sensitivity, specificity

def optimize_threshold(y_true, y_pred_probs):
    """
    Find the threshold that maximizes the G-mean for binary classification.
    """
    best_threshold = -1
    best_gmean = -1
    thresholds = np.linspace(0, 0.2, 201)

    for threshold in thresholds:
        sensitivity, specificity = sen_spec(y_true, y_pred_probs, threshold)
        gmean = np.sqrt(sensitivity * specificity)

        if gmean > best_gmean:
            best_gmean = gmean
            best_threshold = threshold

    return best_threshold, best_gmean

In [7]:
results = {"state": [],
           "threshold": [],
           "acc": [],
           "precision": [],
           "recall": [],
           "f1-score": []}

for state, cp_path in tqdm(trained_cps, total=len(trained_cps)):
    shhs2_records = all_record[all_record['Record'].str.startswith('shhs2')]

    train_records, test_records = train_test_split(shhs2_records, test_size=0.3, 
                                                    random_state=state, 
                                                    stratify=shhs2_records['ahi_label']) # should use AHI
    test_records = pd.concat([test_records, all_record[all_record['Record'].str.startswith('shhs1')]])

    train_records, validation_records = train_test_split(train_records, test_size=0.2, 
                                                    random_state=state, 
                                                    stratify=train_records['ahi_label']) # should use 
    
    
    train_set, _ = load_tfrecord_dataset(train_records['Path'].tolist(), batch_size=1024, shuffle=False)
    test_set, _ = load_tfrecord_dataset(test_records['Path'].tolist(), batch_size=1024, shuffle=False)
    
    model = keras.models.load_model(cp_path)
    
    y_train_true, y_train_pred_prob = get_true_pred(model, train_set, batch_size=1024, verbose=False)
    y_test_true, y_test_pred_prob = get_true_pred(model, test_set, batch_size=1024, verbose=False)
    
    
    # best_threshold, best_gmean = optimize_threshold(y_train_true.flatten(), y_train_pred_prob.flatten())
    best_threshold = 0.1
    y_test_pred = (y_test_pred_prob >= best_threshold).astype(int)
    
    metrics = classification_report(y_test_true, y_test_pred, output_dict=True)
    
    results['state'].append(state)
    results['threshold'].append(best_threshold)
    results['acc'].append(metrics['accuracy'])
    results['precision'].append(metrics['1']['precision'])
    results['recall'].append(metrics['1']['recall'])
    results['f1-score'].append(metrics['1']['f1-score'])

results = pd.DataFrame(results)
results

  0%|          | 0/25 [00:00<?, ?it/s]I0000 00:00:1733284708.008077  115254 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6808 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5
2024-12-04 10:58:28.502636: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:370] TFRecordDataset `buffer_size` is unspecified, default to 262144
I0000 00:00:1733284708.531311  115254 cuda_dnn.cc:529] Loaded cuDNN version 90300
2024-12-04 10:58:38.194638: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-12-04 10:58:47

,state,threshold,acc,precision,recall,f1-score
0,255,0.1,0.956235,0.344092,0.520522,0.414306
1,445,0.1,0.954914,0.331630,0.506262,0.400748
2,411,0.1,0.954494,0.330284,0.513832,0.402102
3,818,0.1,0.957149,0.353891,0.518434,0.420644
4,698,0.1,0.958270,0.357391,0.500679,0.417071
5,529,0.1,0.952225,0.323909,0.544088,0.406073
6,442,0.1,0.955462,0.335902,0.503850,0.403081
7,968,0.1,0.958875,0.367034,0.502172,0.424098
8,743,0.1,0.952747,0.329272,0.553263,0.412843
9,578,0.1,0.951731,0.320662,0.545913,0.404013
